In [ ]:
import pandas as pd

L = '10' # K value of K-mers
fastq_file = L + '.reads.fastq' # Reads the FASTA input file
DBSM = pd.read_parquet('data/' + L + '.DBSM.parquet') # Reads file made by the indexer
indices = pd.read_pickle('data/' + L + '.indices.pkl')

In [ ]:
# SAM Header section
QNAMEs = []
FLAGs = [] #TODO
RNAMEs = [] #chromosome name
POSs = []
MAPQs = []
CIGARs = []
RNEXTs = []
PNEXTs = []
TLENs = []
SEQs = []
QUALs = []

read = []
n = 0
with open(fastq_file, 'r') as f:
    for line in f:
        if len(read) != 4:
            read.append(line.strip())
        else:
            qNAME = read[0]
            while qNAME.startswith('@'):
                qNAME = qNAME[1:]
            SEQ = read[1]
            QUAL = read[3]
            cigars = DBSM['Matches'].get(SEQ, None)
            print(SEQ)
            print(cigars) #prints
            try:
                if cigars == None:
                    cigars = ['*']
            except:
                cigars = list(cigars)
            for CIGAR in cigars:
                FLAG = '0' #consider changing
                RNAME = 'NC_023421.1' # consider changing
                PNEXT = '0' #for single-end
                RNEXT = '*' #for single-end
                TLEN = '0' #for single-end
                if len(cigars) == 1:
                    MAPQ = 60
                else:
                    MAPQ = 0
                    POS = '0'
                if CIGAR[0] != '*':
                    positions = indices['Indices'].get(CIGAR[0], None)
                    CIGAR = CIGAR[1]
                    for POS in positions:
                        POS += 1
                        print(POS)
                        print(open('NC_023421.sequence', 'r').read()[int(POS): int(POS) + int(L) - 1]) #prints
                        print('\n') #prints
                        QNAME = qNAME + ':' + str(n)
                        n += 1
                        print(QNAME, FLAG, RNAME, RNEXT, POS, MAPQ, CIGAR, PNEXT, TLEN, SEQ, QUAL)
                        QNAMEs.append(QNAME)
                        FLAGs.append(FLAG)
                        RNAMEs.append(RNAME)
                        RNEXTs.append(RNEXT)
                        POSs.append(POS)
                        MAPQs.append(MAPQ)
                        CIGARs.append(CIGAR)
                        PNEXTs.append(PNEXT)
                        TLENs.append(TLEN)
                        SEQs.append(SEQ)
                        QUALs.append(QUAL)
            read = [line.strip()]

df = pd.DataFrame()
df['QNAME'] = QNAMEs
df['FLAG'] = FLAGs
df['RNAME'] = RNAMEs
df['POS'] = POSs
df['MAPQ'] = MAPQs
df['CIGAR'] = CIGARs
df['RNEXT'] = RNEXTs
df['PNEXT'] = PNEXTs
df['TLEN'] = TLENs
df['SEQ'] = SEQs
df['QUAL'] = QUALs
df.sort_values(['POS'], inplace = True)
df.to_csv(L + '.aligned.sam', sep = '\t', index = False, header = False)
df